In [ ]:
import pandas as pd
df = pd.read_csv('../data/hansard_speeches_processed.csv')
df = df.sample(n=50000, random_state = 42)

import sys
sys.path.append('../gtm/')
from corpus import GTMCorpus
from gtm import GTM

# Create a GTMCorpus object
train_dataset = GTMCorpus(
    df, 
    prevalence = "~ party", 
    content = "~ party"
)

# Train the model
tm = GTM(
    train_dataset, 
    doc_topic_prior='dirichlet',
    update_prior=True,
    decoder_type='sage',
    decoder_estimate_interactions=True,
    num_epochs=10
)

In [ ]:
# Most common words
import torch
from torch.nn import functional as F
num_words = 8
gamma = tm.AutoEncoder.beta.bias
top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
[tm.id2token[a] for a in top_words]

In [ ]:
# Labour Party deviation (for the whole corpus)
from torch.nn import functional as F
gamma = tm.AutoEncoder.beta_c.weight[:,1:2]
top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
[tm.id2token[a] for a in top_words]

In [ ]:
# Labour Party deviation (for each topic)
for k in range(20):
    gamma = tm.AutoEncoder.beta_ci.weight[:,k*2+1]
    top_words = torch.topk(gamma, k=num_words, dim=0).indices.cpu().numpy().flatten()
    print([tm.id2token[a] for a in top_words])